In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import necessary MongoDB modules
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

# FIX ME - change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import CRUD

# Define MongoDB schema validation
schema = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["age", "breed", "name"],
        "properties": {
            "age": {
                "bsonType": "int",
                "minimum": 0,
                "description": "must be an integer and is required"
            },
            "breed": {
                "bsonType": "string",
                "description": "must be a string and is required"
            },
            "name": {
                "bsonType": "string",
                "description": "must be a string and is required"
            }
        }
    }
}

# MongoDB connection and authentication setup
username = "aacuser"
password = 'SHNU2024'
HOST = 'nv-desktop-services.apporto.com'
PORT = 30703
DB = 'AAC'
COL = 'animals'
connection_options = "?directConnection=true&appName=mongosh+1.8.0"

# Connection URL with credentials
connection_url = f'mongodb://{username}:{password}@{HOST}:{PORT}/{connection_options}'

# Connect to MongoDB and set up schema validation
client = MongoClient(connection_url)
db = client[DB]

# Create collection with schema validation
try:
    db.create_collection(COL, validator=schema)
except Exception as e:
    print(f"Collection creation error: {e}")

collection = db[COL]

# Implement role-based access control
def set_role_based_access_control():
    db.command("createRole", "readWriteRole", privileges=[
        {
            "resource": {"db": DB, "collection": ""},
            "actions": ["find", "insert", "update", "remove"]
        }
    ], roles=[])

    db.command("createUser", "readonly_user", pwd="readonly_password", roles=["readWriteRole"])

set_role_based_access_control()

# Use the CRUD module for database operations
db = CRUD(connection_url, db_name=DB, collection_name=COL)

# Read data from MongoDB and create DataFrame
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# Dashboard Layout / View
app = JupyterDash(__name__)

# Add Grazioso Salvare’s logo
image_filename = 'my-image.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Place the HTML image tag in the app.layout code
app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    html.Div([
        # Add code for interactive filtering options (Radio buttons, dropdown, checkboxes, etc.)
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'ALL'},
                {'label': 'Dog', 'value': 'Dog'},
                {'label': 'Cat', 'value': 'Cat'},
                {'label': 'Other', 'value': 'Other'}
            ],
            value='ALL'
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Set up features for interactive data table
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

# Interaction Between Components / Controller
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    query = {}
    if filter_type == 'Dog':
        query = {'type': 'Dog'}
    elif filter_type == 'Cat':
        query = {'type': 'Cat'}
    elif filter_type == 'Other':
        query = {'type': {'$nin': ['Dog', 'Cat']}}
    
    data = pd.DataFrame.from_records(db.read(query)).to_dict('records')
    return data

# Display the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return "No data to display"
    
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Preferred Animals')
    return dcc.Graph(figure=fig)

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return "No data to display"
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])
                        ])
                    ]
                )
            ]
        )
    ]

app.run_server(debug=True)
